<a href="https://colab.research.google.com/github/petkimani/Financial-Inclusion/blob/main/Streamlit_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Title
st.title("Financial Inclusion Predictor")
st.markdown("Predict whether an individual is likely to have a bank account")

# Load and preprocess data (in production, use preprocessed data or pipeline)
df = pd.read_csv("Financial_inclusion_dataset.csv")
df.drop(["country", "year", "uniqueid"], axis=1, inplace=True)

# Drop NA and handle outliers (minimal version for deploy)
df.dropna(inplace=True)

# Encode target first
df['bank_account'] = df['bank_account'].map({'Yes': 1, 'No': 0})
df['location_type'] = df['location_type'].map({'Rural': 0, 'Urban': 1})
df['cellphone_access'] = df['cellphone_access'].map({'Yes': 1, 'No': 0})
df['gender_of_respondent'] = df['gender_of_respondent'].map({'Male': 1, 'Female': 0})

le = LabelEncoder()
df['relationship_with_head'] = le.fit_transform(df['relationship_with_head'])
df['marital_status'] = le.fit_transform(df['marital_status'])
df['education_level'] = le.fit_transform(df['education_level'])
df['job_type'] = le.fit_transform(df['job_type'])

X = df.drop('bank_account', axis=1)
y = df['bank_account']

# Train model (in production, load a saved model)
model = DecisionTreeClassifier(max_depth=4, criterion='gini', min_samples_split=10, random_state=42)
model.fit(X, y)

# Input form
st.sidebar.header("Enter individual's data")

age = st.sidebar.slider("Age", min_value=16, max_value=100, value=30)

household_size = st.sidebar.slider("Household Size", 1, 20, 4)

location_type = st.sidebar.radio("Location Type", ['Rural', 'Urban'])
cellphone_access = st.sidebar.radio("Has Cellphone Access", ['Yes', 'No'])
gender = st.sidebar.radio("Gender", ['Male', 'Female'])

relationship = st.sidebar.selectbox("Relationship with Head", df['relationship_with_head'].unique())
marital_status = st.sidebar.selectbox("Marital Status", df['marital_status'].unique())
education = st.sidebar.selectbox("Education Level", df['education_level'].unique())
job_type = st.sidebar.selectbox("Job Type", df['job_type'].unique())

# Encode inputs
input_dict = {
    'age_of_respondent': age,
    'household_size': household_size,
    'location_type': 1 if location_type == 'Urban' else 0,
    'cellphone_access': 1 if cellphone_access == 'Yes' else 0,
    'gender_of_respondent': 1 if gender == 'Male' else 0,
    'relationship_with_head': le.transform([relationship])[0],
    'marital_status': le.transform([marital_status])[0],
    'education_level': le.transform([education])[0],
    'job_type': le.transform([job_type])[0]
}

input_df = pd.DataFrame([input_dict])

# Predict
if st.button("Predict"):
    prediction = model.predict(input_df)[0]
    result = "✅ Likely to have a bank account" if prediction == 1 else "❌ Unlikely to have a bank account"
    st.success(result)
